In [5]:
from daq import daqDevice
import daqh
import numpy as np
import time
import serial
import sys
import os
sys.path.append('C:\software_git_repos\cofe-python-analysis-tools\utils_meinhold')
sys.path.append('C:\software_git_repos\cofe-python-analysis-tools\utils_zonca')
sys.path.append('C:\software_git_repos\cofe-python-analysis-tools\utils_zonca\demod')
import cofe_util as cu
sys.path.append('C:\\Anaconda3\\envs\\py27\\Scripts')

In [2]:
def get_data(nchan=4,freq=10,nseconds=5,comment='None',alerts=[58,59,60,118,119,120,178,179,180,238,239,240,298,299,300]):
    """
    function to simply aquire nchan a/d channels at rate freq
    for nseconds seconds
    """
    
    #outdata=np.zeros([nchan,nscans],dtype=float)
    dev=daqDevice('DaqBoard3001USB')
    gains=[]
    flags=[]
    chans=[]
    if nchan > 8:
        uchan=nchan-8
        for i in range(8):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
        for i in range(uchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(256+i)   #HERE is the famous fix where DaqX refs upper level dif channels!
    elif nchan<9:      
        for i in range(nchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
    acqmode = daqh.DaamNShot
    dev.AdcSetAcq(acqmode, postTrigCount = nseconds*freq)
    dev.AdcSetScan(chans,gains,flags)
    dev.AdcSetFreq(freq)
    #use the driver buffer here user buffer was very limited (the way I tried anyway) 
    transMask = daqh.DatmUpdateBlock|daqh.DatmCycleOn|daqh.DatmDriverBuf

    buf=dev.AdcTransferSetBuffer(transMask, np.uint(nseconds*freq*nchan))
    #bufMask is for transferring the buffer
    bufMask = daqh.DabtmOldest | daqh.DabtmRetAvail

    timestart = (time.time())
    timenotify = timestart + 5

    dev.SetTriggerEvent(daqh.DatsImmediate,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStartEvent)
    dev.SetTriggerEvent(daqh.DatsScanCount,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStopEvent)
    dev.AdcTransferStart()
    dev.AdcArm()
    
    while True:
        
        
        alertscopy=alerts[:]
        timenotify = checkAlerts(timenotify, timestart, alerts,alertscopy)        
        
        stat = dev.AdcTransferGetStat()
        active = stat['active']
        if not (active & daqh.DaafAcqActive):
            break
    dev.AdcDisarm()
    outdata,ret=dev.AdcTransferBufData(nseconds*freq, nchan,bufMask)
    
    outdata=np.array(outdata,dtype=float)
    outdata=(outdata-2**15)*20./2**16
    outdata=np.transpose(np.reshape(outdata,[nseconds*freq,nchan]))
    print "Finished collecting data\n----------------------"
    dev.Close()
    

    
    return outdata

In [2]:
def get_date_filename():
    now=time.localtime()[0:6]
    #dirfmt = "c:\\cofe\\ground_data\\testdata\\%4d_%02d_%02d"
    dirfmt = "c:\\cofe\\ground_data\\testdata\\%4d_%02d_%02d"
    dirname = dirfmt % now[0:3]
    filefmt = "%02d_%02d_%02d.dat"
    filename= filefmt % now[3:6]
    ffilename=os.path.join(dirname,filename)
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    return(ffilename)

In [6]:
d300k=getdata(nchan=2,freq=10000,nseconds=10)


NameError: name 'getdata' is not defined

In [26]:
f='l:\PROGRAMS\ATMOS32\summit_5_200_5gm.dat'
z5=genfromtxt(f,skip_header=6,names=True,usecols=[0,1,2])

IOError: l:\PROGRAMS\ATMOS32\summit_5_200_5gm.dat not found.

In [25]:
plot(jnk['Frequency'],jnk['AtmosBrig'],label='45deg,5gm')
ylabel('Atmospheric Brightness, [Krj]')
xlabel('Frequency, [GHz]')
legend()

In [11]:
figure()
plot(d1m[:100000]/1e3,label='1 Mohm')
plot(dterm[:100000]/1e4,label='50 ohm term')
legend()

In [21]:
zterm=util.nps(dterm/1.e4,10000,.01)
z1m=util.nps(d1m/1.e3,10000,.01)

In [29]:
Rn=(0.13*np.sqrt(1e6)/np.sqrt(2))

In [31]:
Rn

91.923881554251167

In [43]:
figure()
plot(zterm[0],1e9*sqrt(zterm[1]),label='50 Ohm term')
plot(z1m[0],1e9*sqrt(z1m[1]),label='1 Mohm')
hlines(Rn,min(z1m[0]),max(z1m[0]),color='r',lw=2,zorder=3)
ylabel('ASD, [nV $\sqrt{sec}$ ]')
xlabel('Frequency, [Hz]')
yscale('log')
xscale('log')
legend()